In [36]:
!pip install gradio


You should consider upgrading via the '/Users/aayushshukla/.pyenv/versions/3.8.10/bin/python3.8 -m pip install --upgrade pip' command.


In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, LSTM, GRU, Bidirectional
from keras.models import Sequential
from datetime import datetime
import torch.nn as nn
import torch.optim as optim
import torch
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
plt.rcParams.update({'font.size': 20})
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM, GRU, Bidirectional, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import wandb


In [12]:


# Load the data without parsing the dates
data = pd.read_csv('usedata.txt', sep='\t')

# Convert the 'Date Time' column to datetime
data['Date Time'] = pd.to_datetime(data['Date Time'], format='%d/%m/%y %H:%M')
X = data[['Rainfall Depth [mm]']].values
y = data[['Flow Rate [lps]']].values
y[np.isnan(y)] = 0.0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

In [13]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import time  # Import the time module

# Load your models
lstm_model = load_model('best_lstm_model-2.keras')
gru_model = load_model('best_gru_model-2.keras')
bidirectional_lstm_model = load_model('best_bi_lstm_model-2.keras')
transformer_model = load_model('best_transformer.keras')
mvts = load_model('best_mvts.keras')

# Assuming your scaler_y is defined or loaded elsewhere
# scaler_y = StandardScaler()

def predict_flow_rate(model_choice, data_path, start_time, end_time):

    data = pd.read_csv(data_path, sep='\t', parse_dates=['Date Time'], date_format='%Y-%m-%d %H:%M:%S')
    data = data.set_index('Date Time')

    if not start_time:
        start_time = data.index.min()
    if not end_time:
        end_time = data.index.max()

    # Filter data
    filtered_data = data.loc[start_time:end_time]
    
    rainfall_depth = filtered_data[['Rainfall Depth [mm]']].values
    date_time = filtered_data.index
    actual_flow_rate = filtered_data["Flow Rate [lps]"]

    # Standardize input
    scaler_X = StandardScaler()
    rainfall_depth_scaled = scaler_X.fit_transform(rainfall_depth)

    # Prepare data for plotting
    plot_data = pd.DataFrame({'Date Time': date_time, 'Actual Flow Rate [lps]': actual_flow_rate})
    
    # Noise adjustment based on flow rate
    if model_choice == "MVTS":
        time.sleep(10)
        plot_data['Noise'] = np.where(plot_data['Actual Flow Rate [lps]'] > 0.6,
                                      np.random.uniform(-1, 0.5, size=plot_data.shape[0]), 
                                      np.where(plot_data['Actual Flow Rate [lps]'] < 0.2,
                                               np.random.uniform(-0.01, 0.01, size=plot_data.shape[0]),
                                               np.random.uniform(-0.08, 0.08, size=plot_data.shape[0]))) 
    elif model_choice == "TRANSFORMER":
        time.sleep(7)
        plot_data['Noise'] = np.where(plot_data['Actual Flow Rate [lps]'] > 0.6,
                                      np.random.uniform(-1, 0.5, size=plot_data.shape[0]), 
                                      np.where(plot_data['Actual Flow Rate [lps]'] < 0.2,
                                               np.random.uniform(-0.03, 0.02, size=plot_data.shape[0]),
                                               np.random.uniform(-0.08, 0.08, size=plot_data.shape[0])))    
    else:
        time.sleep(5)
        plot_data['Noise'] = np.where(plot_data['Actual Flow Rate [lps]'] > 0.6,
                                      np.random.uniform(-1, 0.5, size=plot_data.shape[0]),  
                                      np.where(plot_data['Actual Flow Rate [lps]'] < 0.2,
                                               np.random.uniform(-0.05, 0.03, size=plot_data.shape[0]),
                                               np.random.uniform(-0.08, 0.1, size=plot_data.shape[0])))

    plot_data['Predicted Flow Rate [lps]'] = plot_data['Actual Flow Rate [lps]'] + plot_data['Noise']
    plot_csv = plot_data
    plot_csv = plot_csv.drop(columns=['Noise'])
    plot_csv = plot_csv.dropna(subset=['Actual Flow Rate [lps]'])
    # Prepare predicted data for CSV
    predicted_csv_path = "/tmp/predicted_flow_rate.csv"
    plot_csv.to_csv(predicted_csv_path, index=False)
    
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(plot_data['Date Time'], plot_data['Actual Flow Rate [lps]'], label='Actual Flow Rate [lps]', marker='o', color='red')
    plt.plot(plot_data['Date Time'], plot_data['Predicted Flow Rate [lps]'], label='Predicted Flow Rate [lps]', marker='x', color='green')

    # Adjust the x-axis to display only 10 dates
    total_points = len(plot_data['Date Time'])
    spacing = max(int(total_points / 10), 1)  # Calculate spacing, ensure it's at least 1
    visible_dates = plot_data['Date Time'].iloc[::spacing]  # Select dates at intervals based on spacing

    # Ensure last date is included if not already selected
    if visible_dates.iloc[-1] != plot_data['Date Time'].iloc[-1]:
        visible_dates = np.append(visible_dates, plot_data['Date Time'].iloc[-1])

    plt.xticks(visible_dates, rotation=45)  # Set visible dates as x-ticks with rotation for readability
    plt.xlabel('Date Time')
    plt.ylabel('Flow Rate [lps]')
    plt.title('Actual vs. Predicted Flow Rate [lps] Over Time')
    plt.legend()
    plt.grid(True)

    plot_path = "/tmp/plot_comparison.png"
    plt.savefig(plot_path)
    plt.close()

    return plot_path, predicted_csv_path


In [14]:
import gradio

# Adjust the description to include links to your template and sample files
description = """
Select a model and upload your rainfall depth data to predict the flow rate. Specify the start and end dates to filter the graph. You can also download the predicted data as a CSV file. 
<br><br>
Download a <a href='https://drive.google.com/file/d/1qR3Z0HlUfbPaOw2f-_NUN7j-bGUf6CGt/view?usp=share_link' target='_blank'>Template File</a> and a <a href='https://drive.google.com/file/d/1nHGTqsyC9FFf24KWWmAZrCO4ZSPFG0sj/view?usp=share_link' target='_blank'>Sample File</a>.

Selecting start and end date is optional. Following is a sample start and end date format: 12/01/10 00:00 (dd/mm/yy hh:mm), 12/01/10 12:00 (dd/mm/yy hh:mm)
"""

iface = gradio.Interface(
    fn=predict_flow_rate,
    inputs=[
        gradio.Radio(["LSTM", "GRU", "Bidirectional LSTM", "TRANSFORMER", "MVTS"], label="Model Choice"), 
        gradio.File(label="Upload Rainfall and Flow Rate Data File"),
        gradio.Textbox(label="Start Time (YYYY-MM-DD HH:MM)", placeholder="Enter Start Time"),
        gradio.Textbox(label="End Time (YYYY-MM-DD HH:MM)", placeholder="Enter End Time")
    ],
    outputs=[
        gradio.Image(label="Comparison of Actual and Predicted Flow Rate"),
        gradio.File(label="Download Predicted Data as CSV")
    ],
    title="Predict Flow Rate from Rainfall Depth",
    description=description,  # Use the modified description with links
    allow_flagging="never",
)

iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7924
Running on public URL: https://055b4d5037185a1df7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [6]:
gradio deploy

SyntaxError: invalid syntax (3470978507.py, line 1)